In [4]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [18]:
mp_model = mp_hands.Hands(
static_image_mode=True, # only static images
max_num_hands=2, # max 2 hands detection
min_detection_confidence=0.5) # detection confidence

image = cv2.imread(r"C:\Users\Maxime.ALTER\ProjetIA\ProjetIA\Dev_tib\photos_asl\asl_alphabet_train\asl_alphabet_train\A\A1.jpg")

image = cv2.flip(image, 1)

results = mp_model.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
print(results.multi_handedness)

print(results.multi_hand_world_landmarks)

[classification {
  index: 1
  score: 0.9701958
  label: "Right"
}
]
[landmark {
  x: -0.005012445
  y: 0.079174094
  z: 0.038753867
}
landmark {
  x: -0.029279085
  y: 0.056406274
  z: 0.025241025
}
landmark {
  x: -0.04239887
  y: 0.024935272
  z: 0.02222852
}
landmark {
  x: -0.047755625
  y: -0.0047134096
  z: 0.0107836155
}
landmark {
  x: -0.04828499
  y: -0.03157403
  z: 0.004811785
}
landmark {
  x: -0.026437767
  y: -0.006444908
  z: 0.007553593
}
landmark {
  x: -0.03161918
  y: -0.011089148
  z: -0.016089205
}
landmark {
  x: -0.0359
  y: 0.0074062413
  z: -0.012392311
}
landmark {
  x: -0.03887099
  y: 0.030950293
  z: 0.00024322786
}
landmark {
  x: -0.0030758767
  y: -0.0036876271
  z: 0.003736571
}
landmark {
  x: -0.014499483
  y: -0.009030657
  z: -0.028172037
}
landmark {
  x: -0.023553038
  y: 0.018232737
  z: -0.028075827
}
landmark {
  x: -0.021120917
  y: 0.039500203
  z: -0.007766008
}
landmark {
  x: 0.017373614
  y: 0.0018610754
  z: -0.0045526302
}
landmark {


In [24]:
image_height, image_width, c = image.shape # get image shape
# iterate on all detected hand landmarks

for hand_landmarks in results.multi_hand_landmarks:
      # we can get points using mp_hands
      print(f'Ring finger tip coordinates: (',
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].x * image_width},'
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y * image_height}'
          )

Ring finger tip coordinates: ( 106.80738687515259,95.39629220962524


In [13]:
for hand_landmarks in results.multi_hand_landmarks:
    mp_drawing.draw_landmarks(
        image, # image to draw
        hand_landmarks, # model output
        mp_hands.HAND_CONNECTIONS, # hand connections
        mp_drawing_styles.get_default_hand_landmarks_style(),
        mp_drawing_styles.get_default_hand_connections_style())

In [3]:
# For static images:
IMAGE_FILES = []
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print handedness and draw hand landmarks on the image.
    print('Handedness:', results.multi_handedness)
    if not results.multi_hand_landmarks:
      continue
    image_height, image_width, _ = image.shape
    annotated_image = image.copy()
    for hand_landmarks in results.multi_hand_landmarks:
      print('hand_landmarks:', hand_landmarks)
      print(
          f'Index finger tip coordinates: (',
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
      )
      mp_drawing.draw_landmarks(
          annotated_image,
          hand_landmarks,
          mp_hands.HAND_CONNECTIONS,
          mp_drawing_styles.get_default_hand_landmarks_style(),
          mp_drawing_styles.get_default_hand_connections_style())
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))
    # Draw hand world landmarks.
    if not results.multi_hand_world_landmarks:
      continue
    for hand_world_landmarks in results.multi_hand_world_landmarks:
      mp_drawing.plot_landmarks(
        hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)


In [ ]:
# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()